In [1]:
!pip install playsound==1.2.2


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import cv2
import torch
import numpy as np
from google.colab import files
from IPython.display import display, Image

# You can use the torchvision library for pre-trained models
from torchvision import models, transforms

# Assuming you're using a custom model, ensure it is uploaded to Colab
# Upload the model file if not using a pre-trained model
#uploaded = files.upload()

# Load your model (adjust 'your_model.pth' to the correct file name)
model_path = "/content/gdrive/MyDrive/Food-Spoilage-Detection-System/food_classification2.model/saved_model.pb"
model = torch.load(model_path, map_location=torch.device('cpu'))
model.eval()  # Set the model to evaluation mode



<ipython-input-4-61218126ef1f>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=torch.device('cpu'))


UnpicklingError: invalid load key, '\x08'.

In [ ]:
# Upload a video file
uploaded = files.upload()

# Access the video file
video_path = next(iter(uploaded.keys()))

# Initialize video capture
cap = cv2.VideoCapture(video_path)

# Define a transformation pipeline (depends on your model's requirements)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define label colors
colors = {"good": (0, 255, 0), "bad": (0, 0, 255)}

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to the format expected by the model
    input_tensor = transform(frame)
    input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

    # Get model predictions
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)

    # Assume 0 = good, 1 = bad
    label = "good" if predicted.item()
